In [1]:
import requests
import pandas as pd
import numpy as np

In [2]:
api_key = "x"
username = "Friskis1"
MatchNumber = "5"

In [3]:
api_AccountURL = "https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-name/" + username
print(api_AccountURL)

api_AccountURLandAPI = api_AccountURL + '?api_key=' + api_key
print(api_AccountURLandAPI)

responseAccount = requests.get(api_AccountURLandAPI)
player_info = responseAccount.json()
# print(player_info)

puuid = player_info['puuid']
# print(puuid)

api_MatchHistory = "https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/" + puuid + "/ids?start=0&count=" + MatchNumber
# print(api_MatchHistory)

api_MatchHistoryAPI = api_MatchHistory + '&api_key=' + api_key 
# print(api_MatchHistoryAPI)

responseMatchHistory = requests.get(api_MatchHistoryAPI)
match_ids = responseMatchHistory.json()
print(match_ids)

https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-name/Friskis1
https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-name/Friskis1?api_key=RGAPI-45aa22fd-1a67-4cdc-bb2b-0bb815ea7d02
['EUW1_6465952785', 'EUW1_6465930302', 'EUW1_6465699055', 'EUW1_6465599981', 'EUW1_6465540349']


In [4]:
def calculate_combined_gold(participants):
    total_gold = 0
    for participant in participants:
        total_gold += participant['goldEarned']
    return total_gold

In [46]:
for i in range(int(MatchNumber)):
    api_MatchData = "https://europe.api.riotgames.com/lol/match/v5/matches/" + match_ids[i] + '?api_key=' + api_key
    # print(api_MatchData)

    responseMatchData = requests.get(api_MatchData)
    match_data = responseMatchData.json()

    # Calculate combined gold of the first 5 players
    team1_participants = match_data['info']['participants'][:5]
    team1_gold = calculate_combined_gold(team1_participants)
    # print("Combined gold of Blue side players:", team1_gold)

    # Calculate combined gold of the last 5 players
    team2_participants = match_data['info']['participants'][-5:]
    team2_gold = calculate_combined_gold(team2_participants)
    # print("Combined gold of Red side players:", team2_gold)
    
    # print("Blue side won:", match_data['info']['participants'][0]['win'])

    if match_data['info']['participants'][0]['win']:
        winning_team_gold = team1_gold
        lossing_team_gold = team2_gold
    else:
        winning_team_gold = team2_gold
        lossing_team_gold = team1_gold

    # print("Combined gold winning team:", winning_team_gold)
    # print("Combined gold lossing team:", lossing_team_gold)
    # print("Gold difference between winning and lossing:", winning_team_gold - lossing_team_gold)

    # print("")

In [ ]:
https://europe.api.riotgames.com/lol/match/v5/matches/EUW1_6465540349/timeline

In [6]:
api_MatchTimeline = "https://europe.api.riotgames.com/lol/match/v5/matches/" + match_ids[3] + '/timeline'+ '?api_key=' + api_key 
print(api_MatchTimeline)

responseMatchTimeline = requests.get(api_MatchTimeline)
match_timeline = responseMatchTimeline.json()

https://europe.api.riotgames.com/lol/match/v5/matches/EUW1_6465599981/timeline?api_key=RGAPI-45aa22fd-1a67-4cdc-bb2b-0bb815ea7d02


In [47]:
team_1 = {"Gold": 0, "Level": 0, "Minions": 0, "Jungle_minions": 0}
team_2 = {"Gold": 0, "Level": 0, "Minions": 0, "Jungle_minions": 0}

for i in range(1, 6): 
                           team_1["Gold"] = team_1["Gold"] + match_timeline["info"]["frames"][14]["participantFrames"][f"{i}"]["totalGold"]
                           team_1["Level"] = team_1["Level"] + match_timeline["info"]["frames"][14]["participantFrames"][f"{i}"]["level"]
                           team_1["Minions"] = team_1["Minions"] + match_timeline["info"]["frames"][14]["participantFrames"][f"{i}"]["minionsKilled"]
                           team_1["Jungle_minions"] = team_1["Jungle_minions"] + match_timeline["info"]["frames"][14]["participantFrames"][f"{i}"]["jungleMinionsKilled"]
                    
for i in range(6, 11): 
                           team_2["Gold"] = team_2["Gold"] + match_timeline["info"]["frames"][14]["participantFrames"][f"{i}"]["totalGold"]
                           team_2["Level"] = team_2["Level"] + match_timeline["info"]["frames"][14]["participantFrames"][f"{i}"]["level"]
                           team_2["Minions"] = team_2["Minions"] + match_timeline["info"]["frames"][14]["participantFrames"][f"{i}"]["minionsKilled"]
                           team_2["Jungle_minions"] = team_2["Jungle_minions"] + match_timeline["info"]["frames"][14]["participantFrames"][f"{i}"]["jungleMinionsKilled"]
                    
                    
#Get mean level for each team  
team_1["Level"] = team_1["Level"]/5
team_2["Level"] = team_2["Level"]/5

#Get gold difference         
team_1["Gold_diff"] = team_1["Gold"] - team_2["Gold"]
team_2["Gold_diff"] = team_2["Gold"] - team_1["Gold"]

#Get info of team win or lose
if match_data["info"]["teams"][0]["win"] == True:
    team_1["Win"] = 1
    team_2["Win"] = 0
else:
    team_1["Win"] = 0
    team_2["Win"] = 1
    
    
team_1_data = {'Gold': [team_1["Gold"]], 'Level': [team_1["Level"]], 'Minions': [team_1["Minions"]], 'Jungle_minions': [team_1["Jungle_minions"]], 'Gold_diff': [team_1["Gold_diff"]], 'Win': [team_1["Win"]]}
team_2_data = {'Gold': [team_2["Gold"]], 'Level': [team_2["Level"]], 'Minions': [team_2["Minions"]], 'Jungle_minions': [team_2["Jungle_minions"]], 'Gold_diff': [team_2["Gold_diff"]], 'Win': [team_2["Win"]]}

team_1_df = pd.DataFrame(team_1_data, index=[0])
team_2_df = pd.DataFrame(team_2_data, index=[0])

combined_df = pd.concat([team_1_df, team_2_df], ignore_index=True)

combined_df.to_csv('team_data.csv', index=False)

In [49]:
import pandas as pd

team_1_data = {'Gold': [team_1["Gold"]], 'Level': [team_1["Level"]], 'Minions': [team_1["Minions"]],
               'Jungle_minions': [team_1["Jungle_minions"]], 'Gold_diff': [team_1["Gold_diff"]], 'Win': [team_1["Win"]]}
team_2_data = {'Gold': [team_2["Gold"]], 'Level': [team_2["Level"]], 'Minions': [team_2["Minions"]],
               'Jungle_minions': [team_2["Jungle_minions"]], 'Gold_diff': [team_2["Gold_diff"]], 'Win': [team_2["Win"]]}

combined_df = pd.concat([pd.DataFrame(team_1_data), pd.DataFrame(team_2_data)], ignore_index=True)

combined_df.to_csv('team_data1.csv', index=False)
